<h1><center> EL7006 - Tarea Nº4 <br />Generative Models:  Variational Autoencoders <br/> </center></h1>
<h3><center>Profesor: Pablo Estévez <br />
Profesor Auxiliar: Jhon Intriago <br />
Ayudantes: Bastián Gamboa, Daniel Neira, Giovanni Castiglioni <br />
<h3><center>Semestre: Primavera 2022 </center></h3>

<h1><center><u>Parte Teórica</u></center></h1>



Como se vió en la clase de VAEs, la función de costo a optimizar está dada por:

$$
\mathcal{L}_{\text{negative-ELBO}}   = \mathbb{E}_{q(x)}[- \mathbb{E}_{q(z|x)}[\log p(x|z)] + D_{KL}(q(z|x)|| p(z)) ] 
$$

Donde $D_{KL}(q(z|x)|| p(z)) = \mathbb{E}_{q(z|x)}[\log q(z|x) - \log p(z)] $. En general las entropías y entropías cruzadas en el término $\mathcal{L}_{\text{negative-ELBO}}$ se obtienen de forma cerrada. Nos centraremos en el término regularizador $D_{KL}(q(z|x)|| p(z))$. 

Para obtener una función de pérdida cerrada en $D_{KL}(q(z|x)|| p(z))$ se realizan ciertas suposiciones. Las más relevantes son:

* La suposición mean field del encoder $q(z|x) = \prod_{j=1}^J q(z_j|x)$. Es decir no existe una correlación entre las variables latentes inferidas. $J$ el número de dimensiones.
* Se asume que $q(z_j|x)$ distribuye normal.
* Se asume una distribución prior $p(z) = \mathcal{N}(0, I)$.

Se obtendrá la función de costo de VAEs partiendo de un caso un poco más general, considerando $q(z|x) = \mathcal{N}(\mu_1, \Sigma_1)$ y $p(z) = \mathcal{N}(\mu_2, \Sigma_2)$ y posteriormente aplicando las suposiciones mencionadas se llegará a la fórmula típica usada en VAEs. Tomar $q(z|x)$ o $p(z)$ como distribuciones un poco más generales le dan mayor flexibilidad al modelo.

Para esto, considere los siguientes pasos: 

* Calcule $\int q(z|x) \log p(z)$. Como está trabajando con matrices le puede servir http://www2.imm.dtu.dk/pubdb/pubs/3274-full.html. Notar que debe obtener una solución matricial para esta entropía cruzada negativa.

     <font size=1 color=red> <b>Escriba su demostración con todos los pasos aquí </b></font>     


* Con el resultado anterior, que debiera estar en términos matriciales, asuma la suposición mean field tanto en $q(z|x)$ como $p(z)$ y llegue a 
$
\frac{1}{2}\sum_{j=1}^J \left( - \log(2\pi\sigma_{2j}^2) - \frac{\sigma^2_{1j}}{\sigma^2_{2j}} - \frac{(\mu_{1j} - \mu_{2j})^2}{\sigma_{2j}^2} \right)
$. Donde para $\mu$ y $\sigma^2$ el primer índice se refiere a la distribución y el segundo a las dimensiones. La demostración para el caso multivariado utilizando mean field se puede encontrar en https://arxiv.org/abs/1611.05148. No se utiliza esta demostración, se incluye por completitud.

    <font size=1 color=red> <b>Escriba su demostración aquí </b></font> 
    
    
* Utilizando la expresión anterior, tomando prior $p(z) = \mathcal{N}(0,I)$ y utilizando la expresión $D_{KL}(q(z|x)|| p(z)) = \mathbb{E}_{q(z|x)}[\log q(z|x) - \log p(z)] $ , llegue a la fórmula cerrada de esta divergencia utilizada en variational autoencoders dada por $\frac{1}{2}\sum_{j = 1}^J \left( \mu(X) + \sigma^2(x) - \log \sigma^2(x) - 1  \right)$.

    <font size=1 color=red> <b>Escriba su demostración aquí </b></font> 



<h1><center><u>Parte Práctica </u></center></h1>

En este parte, usted debe implementar y entrenar un modelo de VAEs. Además deberá comparar el desempeño de estos modelos según la métrica de evaluación solicitada. 


<h2>Preliminares: Google Colab</h2>

Para evitar limitaciones de hardware, ejecute este notebook en Google Colab con GPU. Para ello siga estos pasos:

- Suba este notebook a su cuenta de Google Drive.
- Abra el notebook. Se hará automáticamente en Google Colab.
- En el menú "Entorno de ejecución", seleccione "Cambiar tipo de entorno de ejecución" y en "Acelerador por hardware" seleccione GPU.

<h2>Preliminares: PyTorch</h2>

Por simplicidad, en esta tarea se utilizará PyTorch. Se entrega una estructura general del código a implementar, con el objetivo de no requerir conocimientos profundos del framework utilizado.

<h2> Funcionamiento VAE  </h2>

A continuación se recuerda cómo funciona un Variational Autoencoder.

1. Se samplea de la distribución empírica $q(x)$. (Esto sería el data loader)

2. A través del encoder se obtienen los $\mu$ y los $\log \sigma^2$. En general se parametriza $\log \sigma^2$ en vez de $\sigma$, pues la salida generalmente se toma como lineal y basta tomar la exponencial de $\log \sigma^2$ para obtener $\sigma^2$ y así asegurar que este sea positivo.

3. Se samplea ruido proveniente de una distribución normal $\epsilon \sim \mathcal{N}(0, I)$
4. Se utiliza el truco de [reparametrización](https://arxiv.org/abs/1312.6114) para samplear de $q(z|x)$ y además optimizar sus parametros (los parámetros del encoder). Esto es, el $\tilde{z}$ sampleado de $q(z|x)$ se puede escribir como $\tilde{z} = \mu(x) + \sigma(x) \odot \epsilon $. Este paso en general se utiliza para optimizar el encoder mediante la función de pérdida asociada a la verosimilitud. La divergencia no toma en cuenta cuantos sampleos se hagan ya que se calcula de forma cerrada. En caso que no se calculase en forma cerrada, importaría el número de sampleos de $q(z|x)$. Para la tarea sólo debe samplear un $z(\tilde{x}_i)$ por cada $x_i \sim q(x) $. Esto es lo que se conoce como los sampleos de Monte-Carlo.
5. Se calcula la verosomilitud negativa y se minimiza. Se calcula la divergencia de la parte teórica y se minimiza.


<h2> Deep Convolutional VAE  </h2>

En esta sección, usted deberá implementar una arquitectura muy similar a la pregunta de GANs en la que se utiliza la [DCGAN](https://arxiv.org/pdf/1511.06434v1.pdf). Se utilizará el mismo generador. El encoder utilizado será casi completamente similar al discriminador de la DCGAN.

Alguna de las características de la DCGAN son: 
*   Usa convoluciones sin capas de pooling
*   Usa BatchNormalization tanto en el Generador como en el Discriminador (En este caso no se tiene un Discriminador pero se utilizará un Encoder con las mismas características).
*   No usa capas <i> Fully Connected </i>
*   Usa activaciones ReLU en el Generador, salvo en la última capa donde usa Tanh. En este caso en vez de Tanh debe tener una salida lineal.
*   Usa activaciones LeakyReLU en el Discriminador (en este caso Encoder), salvo en la última capa donde no usa activación

A continuación se importan los paquetes a utilizar.

In [ ]:
from IPython import display
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(0); # Set for testing purposes, please do not change!

In [ ]:
# If you have multiple devices and just want to use one, 
# then uncomment the following two lines:
# i_device = 1
# device = 'cuda:{0}'.format(i_device)
# Otherwise, uncomment the following line:
# device = 'cuda'

<h3> Bloque del Generador </h3>

Para comenzar, deberá implementar los bloques que componen el Generador.
Debido a que la función de activación varía según la capa, se deberá considerar esto al momento de la creación del bloque.

La estructura de cada bloque deberá ser la siguiente: 
*   Una capa deconvolucional, utilizando los parámetros dados
*   Una capa de batchnorm, salvo en la última capa
*   Una activación ReLU después de cada batchnorm
*   Una activacion lineal, sólo para la última capa

In [ ]:
def make_gen_block(input_channels, output_channels, kernel_size=3, stride=2, final_layer=False):
    if not final_layer:
        return nn.Sequential(
        ### YOUR CODE HERE ###
        )
    else:
        return nn.Sequential(
        ### YOUR CODE HERE ###
        )

In [ ]:
def get_noise(n_samples, z_dim, device='cpu'):
    return torch.randn(n_samples, z_dim, device=device)

<h2> Generador</h2>

Ahora construya un Generador formado por 4 bloques, con las siguientes especificaciones:

    1.-block: z_dim          -> hidden_dim * 4 , kernel_size=3, stride=2
    2.-block: hidden_dim * 4 -> hidden_dim * 2 , kernel_size=4, stride=1
    3.-block: hidden_dim * 2 -> hidden_dim     , kernel_size=3, stride=2
    4.-block: hidden_dim     -> im_chan        , kernel_size=4, stride=2


In [ ]:
class Generator(nn.Module):
    def __init__(self, z_dim=10, im_chan=1, hidden_dim=64):
        super(Generator, self).__init__()
        self.z_dim = z_dim

        self.gen = nn.Sequential(
        ### YOUR CODE HERE ###
        )

    def unsqueeze_noise(self, noise):
        return noise.view(len(noise), self.z_dim, 1, 1)

    def forward(self, noise):
        x = self.unsqueeze_noise(noise)
        return self.gen(x)

<h3> Bloque del Encoder</h3>

Procederemos a implementar un bloque para el Encoder.
La estructura de cada bloque deberá ser la siguiente: 

*    Una capa convolucional, utilizando los parámetros dados
*    Una capa de batchnorm, excepto por la última capa
*    Una activacion LeakyReLU de pendiente 0.2, sólo después de las capas de batchnorm

In [ ]:
def make_enc_block(input_channels, output_channels, kernel_size=4, stride=2, final_layer=False):     
        if not final_layer:
            return nn.Sequential(
            ### YOUR CODE HERE ###
            )
        else:
            return nn.Sequential(
            ### YOUR CODE HERE ###
            )

<h2> Encoder </h2>

Ahora construya un Encoder formado por 3 bloques, con las siguientes especificaciones:

    1.-block: im_chan        -> hidden_dim     , kernel_size=4, stride=2
    2.-block: hidden_dim     -> hidden_dim * 2 , kernel_size=4, stride=2
    3.-block: hidden_dim * 2 -> 2*z_dim        , kernel_size=4, stride=2
    
Además tiene que definir la función split_output, que separa la predicción del encoder en los $\mu$ y $\log \sigma^2$.

In [ ]:
class Encoder(nn.Module):
    def __init__(self, z_dim = 2, im_chan=1, hidden_dim=16):
        super(Encoder, self).__init__()
        self.enc = nn.Sequential(
        ### YOUR CODE HERE ###
        )
        
    def split_output(self, features):
        ### YOUR CODE HERE ###
        
    def forward(self, image):
        enc_pred = self.enc(image)
        return self.split_output(enc_pred)

A continuación se definen los parámetros de entrenamiento a utlizar, y una función de utilidad para la inicialización de los pesos.

In [ ]:
# Setting the training parameters
z_dim = 2
display_step = 500
batch_size = 128
lr = 0.0002
beta_1 = 0.5 
beta_2 = 0.999

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

dataloader = DataLoader(
    MNIST('.', download=True, transform=transform),
    batch_size=batch_size,
    shuffle=True)

In [ ]:
gen = Generator(z_dim).to(device)
gen_opt = torch.optim.Adam(gen.parameters(), lr=lr, betas=(beta_1, beta_2))
enc = Encoder().to(device) 
enc_opt = torch.optim.Adam(enc.parameters(), lr=lr, betas=(beta_1, beta_2))

def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.normal_(m.weight, 0.0, 0.02)
    if isinstance(m, nn.BatchNorm2d):
        torch.nn.init.normal_(m.weight, 0.0, 0.02)
        torch.nn.init.constant_(m.bias, 0)
gen = gen.apply(weights_init)
enc = enc.apply(weights_init)

<h3> Funciones de Loss</h3>

Implemente los métodos `get_neg_likelihood_loss` y `get_divergence_loss`. La función `get_neg_likelihood_loss` calcula la verosimilitud negativa en el espacio observable. Esta la debe calcular como un MSE entre la reconstrucción y el dato original como se vio en clases. La función `get_divergence_loss` es la divergencia vista en la parte teórica.

In [ ]:
def get_neg_likelihood_loss(x, x_rec):
    ### YOUR CODE HERE ###
    neg_lik_loss = None
    return neg_lik_loss

def get_divergence_loss(mu, logvar):
    ### YOUR CODE HERE ###
    divergence_loss = None
    return divergence_loss

<h2> Entrenamiento VAE </h2>

Muchas veces en la práctica se usa algo conocido como [$\beta$-VAE](https://openreview.net/forum?id=Sy2fzU9gl) cuya función de costo es muy similar a la función de VAE. La única diferencia es un ponderador en la divergencia como se muestra a continuación:
$$
\mathcal{L}_{\text{negative-ELBO}-\beta-\text{VAE}}   = \mathbb{E}_{q(x)}[- \mathbb{E}_{q(z|x)}[\log p(x|z)] + \beta \cdot D_{KL}(q(z|x)|| p(z)) ] 
$$

A continuación defina una función de entrenamiento de VAE dejando $\beta$ como parámetro libre. En su entrenamiento comente todos los puntos mencionados en  <b>Funcionamiento VAE</b>. Utilice todas las funciones creadas en las secciones anteriores. Puede guiarse por el loop de entrenamiento utilizado en la pregunta de GANs.


<h2> Plot 2-D </h2>

Utilizando la función anterior, varíe $\beta$ tal que la distribución marginal $q(z)$ sea aproximadamente una gaussiana. Para mostrar esto, genere tres plots 2D del espacio latente; uno con $\beta$ bajo, otro con un valor medio y otro con valor alto. ¿Qué ocurre para cada valor de $\beta$?